In [19]:
# ===============================================================
# ML Project — Macro + Market Data
# Pipeline : FRED + WorldBank + OCDE(KEI/DB.NOMICS) → Merge → Weekly → GB
# Output files : CSV
# ===============================================================

# =========================
# 0) Data Import
# =========================
import re, datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import wbdata
import requests
import yfinance as yf
from pathlib import Path

START_DATE = "2010-01-01"
END_DATE   = "2025-09-01"
date_index = pd.date_range(start=START_DATE, end=END_DATE, freq="D")

OUT = Path.home() / "ml_outputs"
OUT.mkdir(parents=True, exist_ok=True)

In [20]:
# ==============================
# 1) FRED — Times Series
# ==============================
fred_series = {
    "CPIAUCSL": "Inflation_CPI",
    "UNRATE": "Unemployment_Rate",
    "FEDFUNDS": "Fed_Funds_Rate",
    "GDPC1": "Real_GDP",
    "T10Y3M": "Yield_Curve_Spread",
    "INDPRO": "Industrial_Production",
    "M2SL": "M2_Money_Supply",
    "PCE": "Personal_Consumption_Expenditures",
    "RECPROUSM156N": "Recession_Probability",
}
fred_data = pdr.DataReader(list(fred_series.keys()), "fred", start=START_DATE, end=END_DATE)
fred_data.rename(columns=fred_series, inplace=True)
fred_data = fred_data.sort_index().reindex(date_index).ffill().bfill()

from IPython.display import display
print("FRED shape:", fred_data.shape)
display(fred_data.head())

FRED shape: (5723, 9)


,Inflation_CPI,Unemployment_Rate,Fed_Funds_Rate,Real_GDP,Yield_Curve_Spread,Industrial_Production,M2_Money_Supply,Personal_Consumption_Expenditures,Recession_Probability
2010-01-01,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-02,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-03,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-04,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-05,217.488,9.8,0.11,16582.71,3.70,89.1897,8477.7,10056.1,0.08


In [21]:
from IPython.display import display
pd.set_option("display.max_rows", 200)   # optionnel

print("FRED 2010 Jan–Mar:")
display(fred_data.loc["2010-01":"2010-03"])

print("FRED — 50 premières lignes:")
display(fred_data.iloc[:50])

print("FRED — 50 dernières lignes:")
display(fred_data.iloc[-50:])

FRED 2010 Jan–Mar:


,Inflation_CPI,Unemployment_Rate,Fed_Funds_Rate,Real_GDP,Yield_Curve_Spread,Industrial_Production,M2_Money_Supply,Personal_Consumption_Expenditures,Recession_Probability
2010-01-01,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-02,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-03,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-04,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-05,217.488,9.8,0.11,16582.71,3.70,89.1897,8477.7,10056.1,0.08
2010-01-06,217.488,9.8,0.11,16582.71,3.79,89.1897,8477.7,10056.1,0.08
2010-01-07,217.488,9.8,0.11,16582.71,3.80,89.1897,8477.7,10056.1,0.08
2010-01-08,217.488,9.8,0.11,16582.71,3.78,89.1897,8477.7,10056.1,0.08
2010-01-09,217.488,9.8,0.11,16582.71,3.78,89.1897,8477.7,10056.1,0.08
2010-01-10,217.488,9.8,0.11,16582.71,3.78,89.1897,8477.7,10056.1,0.08


FRED — 50 premières lignes:


,Inflation_CPI,Unemployment_Rate,Fed_Funds_Rate,Real_GDP,Yield_Curve_Spread,Industrial_Production,M2_Money_Supply,Personal_Consumption_Expenditures,Recession_Probability
2010-01-01,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-02,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-03,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-04,217.488,9.8,0.11,16582.71,3.77,89.1897,8477.7,10056.1,0.08
2010-01-05,217.488,9.8,0.11,16582.71,3.70,89.1897,8477.7,10056.1,0.08
2010-01-06,217.488,9.8,0.11,16582.71,3.79,89.1897,8477.7,10056.1,0.08
2010-01-07,217.488,9.8,0.11,16582.71,3.80,89.1897,8477.7,10056.1,0.08
2010-01-08,217.488,9.8,0.11,16582.71,3.78,89.1897,8477.7,10056.1,0.08
2010-01-09,217.488,9.8,0.11,16582.71,3.78,89.1897,8477.7,10056.1,0.08
2010-01-10,217.488,9.8,0.11,16582.71,3.78,89.1897,8477.7,10056.1,0.08


FRED — 50 dernières lignes:


,Inflation_CPI,Unemployment_Rate,Fed_Funds_Rate,Real_GDP,Yield_Curve_Spread,Industrial_Production,M2_Money_Supply,Personal_Consumption_Expenditures,Recession_Probability
2025-07-14,322.132,4.2,4.33,23770.976,0.01,103.8194,22028.8,20982.7,0.64
2025-07-15,322.132,4.2,4.33,23770.976,0.08,103.8194,22028.8,20982.7,0.64
2025-07-16,322.132,4.2,4.33,23770.976,0.05,103.8194,22028.8,20982.7,0.64
2025-07-17,322.132,4.2,4.33,23770.976,0.06,103.8194,22028.8,20982.7,0.64
2025-07-18,322.132,4.2,4.33,23770.976,0.04,103.8194,22028.8,20982.7,0.64
2025-07-19,322.132,4.2,4.33,23770.976,0.04,103.8194,22028.8,20982.7,0.64
2025-07-20,322.132,4.2,4.33,23770.976,0.04,103.8194,22028.8,20982.7,0.64
2025-07-21,322.132,4.2,4.33,23770.976,-0.03,103.8194,22028.8,20982.7,0.64
2025-07-22,322.132,4.2,4.33,23770.976,-0.06,103.8194,22028.8,20982.7,0.64
2025-07-23,322.132,4.2,4.33,23770.976,-0.01,103.8194,22028.8,20982.7,0.64


In [22]:
# ==============================
# 2) WORLD BANK — USA
# ==============================
wb_indicators = {
    "NY.GDP.MKTP.CD": "GDP_Current_USD",
    "FP.CPI.TOTL.ZG": "Inflation_Annual_Pct",
    "SL.UEM.TOTL.ZS": "Unemployment_Total_Pct",
    "NE.EXP.GNFS.ZS": "Exports_GDP_Pct",
    "NE.IMP.GNFS.ZS": "Imports_GDP_Pct",
    "GC.DOD.TOTL.GD.ZS": "Public_Debt_GDP_Pct",
}
try:
    wb_data = wbdata.get_dataframe(
        wb_indicators, country="US",
        date=(dt.datetime(2010,1,1), dt.datetime(2025,9,1))
    )
except TypeError:
    wb_data = wbdata.get_dataframe(
        wb_indicators, country="US",
        data_date=(dt.datetime(2010,1,1), dt.datetime(2025,9,1))
    )
wb_data.index = pd.to_datetime(wb_data.index, errors="coerce")
wb_data = wb_data.sort_index().reindex(date_index).ffill().bfill()

print("World Bank shape:", wb_data.shape)
display(wb_data.head())


World Bank shape: (5723, 6)


,GDP_Current_USD,Inflation_Annual_Pct,Unemployment_Total_Pct,Exports_GDP_Pct,Imports_GDP_Pct,Public_Debt_GDP_Pct
2010-01-01,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-02,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-03,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-04,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-05,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374


In [24]:
from IPython.display import display
print("World Bank — 2025:")
display(wb_data.loc["2025"])

World Bank — 2025:


,GDP_Current_USD,Inflation_Annual_Pct,Unemployment_Total_Pct,Exports_GDP_Pct,Imports_GDP_Pct,Public_Debt_GDP_Pct
2025-01-01,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-01-02,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-01-03,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-01-04,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-01-05,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
...,...,...,...,...,...,...
2025-08-28,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-29,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-30,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-31,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553


In [25]:
# =================================================
# 3) OCDE (KEI) via DB.NOMICS — CPI & Unemployment
# =================================================
def oecd_kei_to_wide(series_ids):
    base = "https://api.db.nomics.world/v22/series?observations=1&format=csv&series_ids="
    url = base + ",".join(series_ids)
    df = pd.read_csv(url, low_memory=False)

    # trouver colonne période
    pcol = next((c for c in df.columns if c.strip().lower() in {"period","time","time_period","date"}), None)
    if pcol is None:
        raise KeyError(f"Colonne période introuvable: {list(df.columns)[:10]}")
    df = df.rename(columns={pcol: "period"})

    # renommer colonnes → extraire ID entre parenthèses (...(OECD/KEI/...))
    ren = {}
    for c in df.columns:
        if c == "period": continue
        m = re.search(r"\(([^()]+)\)\s*$", str(c))
        ren[c] = (m.group(1) if m else c).upper()
    df = df.rename(columns=ren)

    keep = ["period"] + [s.upper() for s in series_ids]
    missing = [s for s in series_ids if s.upper() not in df.columns]
    if missing:
        # On laisse passer, on créera un fallback
        print("⚠️ Séries OCDE manquantes dans le CSV:", missing)
        keep = [k for k in keep if k in df.columns]
    wide = df[keep].copy()

    # index = fin de mois
    wide.index = pd.PeriodIndex(wide["period"].astype(str), freq="M").end_time
    wide = wide.drop(columns=["period"]).sort_index()
    wide.index.name = "DATE"
    return wide

oecd_series_ids = [
    "OECD/KEI/CPALTT01.USA.ST.M",  # CPI 2015=100, mensuel, s.a.
    "OECD/KEI/LRHUTTTT.USA.ST.M",  # Unemployment %, mensuel, s.a.
]
oecd_data = oecd_kei_to_wide(oecd_series_ids)
oecd_data = oecd_data.reindex(date_index).ffill().bfill()
oecd_data = oecd_data.rename(columns={
    "OECD/KEI/CPALTT01.USA.ST.M": "OECD_CPI_2015idx_USA",
    "OECD/KEI/LRHUTTTT.USA.ST.M": "OECD_Unemp_rate_pct_USA",
})
print("OCDE shape (avant fallback):", oecd_data.shape, "| NA:", int(oecd_data.isna().sum().sum()))


OCDE shape (avant fallback): (5723, 2) | NA: 11446


In [26]:
# ===========================================================
# 3bis) Fallback OCDE depuis FRED si colonnes vides (0 NaN)
# ===========================================================
def fill_oecd_from_fred(oecd_df, fred_df, date_index):
    o = oecd_df.copy()
    changed = []

    # CPI indice 2015=100 à partir de CPIAUCSL (mensuel)
    if "OECD_CPI_2015idx_USA" in o.columns and o["OECD_CPI_2015idx_USA"].isna().all():
        cpi_m = fred_df["Inflation_CPI"].resample("ME").last()
        base_2015 = cpi_m.loc["2015-01-31":"2015-12-31"].mean()
        o["OECD_CPI_2015idx_USA"] = (cpi_m/base_2015*100).reindex(date_index).ffill().bfill()
        changed.append("OECD_CPI_2015idx_USA ← FRED CPIAUCSL (base 2015=100)")

    # Unemployment % à partir de UNRATE (mensuel)
    if "OECD_Unemp_rate_pct_USA" in o.columns and o["OECD_Unemp_rate_pct_USA"].isna().all():
        un_m = fred_df["Unemployment_Rate"].resample("ME").last()
        o["OECD_Unemp_rate_pct_USA"] = un_m.reindex(date_index).ffill().bfill()
        changed.append("OECD_Unemp_rate_pct_USA ← FRED UNRATE")

    if changed:
        print("OCDE fallback:", changed)
    else:
        print("OCDE fallback: aucun")
    return o

oecd_data = fill_oecd_from_fred(oecd_data, fred_data, date_index)
print("OCDE NA après fallback:", int(oecd_data.isna().sum().sum()))


OCDE fallback: ['OECD_CPI_2015idx_USA ← FRED CPIAUCSL (base 2015=100)', 'OECD_Unemp_rate_pct_USA ← FRED UNRATE']
OCDE NA après fallback: 0


In [27]:
oecd_data

,OECD_CPI_2015idx_USA,OECD_Unemp_rate_pct_USA
2010-01-01,91.766411,9.8
2010-01-02,91.766411,9.8
2010-01-03,91.766411,9.8
2010-01-04,91.766411,9.8
2010-01-05,91.766411,9.8
...,...,...
2025-08-28,135.919671,4.2
2025-08-29,135.919671,4.2
2025-08-30,135.919671,4.2
2025-08-31,136.439499,4.3


In [28]:
fred_data

,Inflation_CPI,Unemployment_Rate,Fed_Funds_Rate,Real_GDP,Yield_Curve_Spread,Industrial_Production,M2_Money_Supply,Personal_Consumption_Expenditures,Recession_Probability
2010-01-01,217.488,9.8,0.11,16582.710,3.77,89.1897,8477.7,10056.1,0.08
2010-01-02,217.488,9.8,0.11,16582.710,3.77,89.1897,8477.7,10056.1,0.08
2010-01-03,217.488,9.8,0.11,16582.710,3.77,89.1897,8477.7,10056.1,0.08
2010-01-04,217.488,9.8,0.11,16582.710,3.77,89.1897,8477.7,10056.1,0.08
2010-01-05,217.488,9.8,0.11,16582.710,3.70,89.1897,8477.7,10056.1,0.08
...,...,...,...,...,...,...,...,...,...
2025-08-28,323.364,4.3,4.33,23770.976,-0.04,103.9203,22108.4,21111.9,0.96
2025-08-29,323.364,4.3,4.33,23770.976,0.00,103.9203,22108.4,21111.9,0.96
2025-08-30,323.364,4.3,4.33,23770.976,0.00,103.9203,22108.4,21111.9,0.96
2025-08-31,323.364,4.3,4.33,23770.976,0.00,103.9203,22108.4,21111.9,0.96


In [29]:
wb_data

,GDP_Current_USD,Inflation_Annual_Pct,Unemployment_Total_Pct,Exports_GDP_Pct,Imports_GDP_Pct,Public_Debt_GDP_Pct
2010-01-01,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-02,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-03,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-04,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
2010-01-05,1.504897e+13,1.640043,9.633,12.341355,15.878528,84.964374
...,...,...,...,...,...,...
2025-08-28,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-29,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-30,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553
2025-08-31,2.918489e+13,2.949525,4.106,10.896875,13.991117,114.755553


In [30]:
oecd_data

,OECD_CPI_2015idx_USA,OECD_Unemp_rate_pct_USA
2010-01-01,91.766411,9.8
2010-01-02,91.766411,9.8
2010-01-03,91.766411,9.8
2010-01-04,91.766411,9.8
2010-01-05,91.766411,9.8
...,...,...
2025-08-28,135.919671,4.2
2025-08-29,135.919671,4.2
2025-08-30,135.919671,4.2
2025-08-31,136.439499,4.3


In [39]:
# -----------------------------------------------
# 4) MERGE ALL MACRO DATA Sources (+ saving)
# -----------------------------------------------

macro_data = (
    fred_data
    .join(wb_data, how="outer")
    .join(oecd_data, how="outer")
    .sort_index()
    .ffill()
    .bfill()
)

print("NA total (macro):", int(macro_data.isna().sum().sum()))

# Save files
macro_data.to_csv(OUT / "Macro_AllSources.csv")
fred_data.to_csv(OUT / "Macro_FRED.csv")
wb_data.to_csv(OUT / "Macro_WorldBank.csv")
oecd_data.to_csv(OUT / "Macro_OECD.csv")
print("Saved macro CSVs to:", OUT)

NA total (macro): 0
Saved macro CSVs to: C:\Users\dax_a\ml_outputs


In [40]:
#
# 5) Market Data 
#

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

# --- PARAMETERS ---
START_DATE = "2010-01-01"
END_DATE   = datetime.today().strftime("%Y-%m-%d")

# Utility function to fetch data
def fetch_data(tickers, start=START_DATE, end=None):
    data = yf.download(tickers, start=start, end=end, progress=False, group_by='ticker', auto_adjust=True)

    # Handle structure depending on number of tickers
    if isinstance(data.columns, pd.MultiIndex):
        # MultiIndex when multiple tickers are requested
        data_close = pd.DataFrame()
        for t in tickers:
            # Try to extract 'Adj Close' or fall back to 'Close'
            if ('Adj Close' in data[t].columns):
                data_close[t] = data[t]['Adj Close']
            elif ('Close' in data[t].columns):
                data_close[t] = data[t]['Close']
            else:
                print(f"⚠️ No valid price data found for {t}")
    else:
        # Single ticker case
        col = 'Adj Close' if 'Adj Close' in data.columns else 'Close'
        data_close = data[[col]].rename(columns={col: tickers[0]})

    data_close.dropna(how='all', inplace=True)
    return data_close

In [41]:
index_tickers = {
    "S&P500": "^GSPC",
    "NASDAQ": "^IXIC",
    "DowJones": "^DJI",
    "CAC40": "^FCHI",
    "DAX": "^GDAXI",
    "FTSE100": "^FTSE",
    "Nikkei225": "^N225",
    "HangSeng": "^HSI",
    "MSCIWorld": "URTH"
}

indexes = fetch_data(list(index_tickers.values()))
indexes.columns = index_tickers.keys()
print(indexes.head())
print(indexes.tail())

                 S&P500       NASDAQ      DowJones        CAC40          DAX  \
Date                                                                           
2010-01-04  1132.989990  2308.419922  10583.959961  4013.969971  6048.299805   
2010-01-05  1136.520020  2308.709961  10572.019531  4012.909912  6031.859863   
2010-01-06  1137.140015  2301.090088  10573.679688  4017.669922  6034.330078   
2010-01-07  1141.689941  2300.050049  10606.860352  4024.800049  6019.359863   
2010-01-08  1144.979980  2317.169922  10618.190430  4045.139893  6037.609863   

                FTSE100     Nikkei225      HangSeng  MSCIWorld  
Date                                                            
2010-01-04  5500.299805  10654.790039  21823.279297        NaN  
2010-01-05  5522.500000  10681.830078  22279.580078        NaN  
2010-01-06  5530.000000  10731.450195  22416.669922        NaN  
2010-01-07  5526.700195  10681.660156  22269.449219        NaN  
2010-01-08  5534.200195  10798.320312  22296.7500

In [42]:
# Major Government Bond ETFs and Yield Proxies
bond_tickers = {
    "US10Y": "^TNX",       # 10-Year Treasury Yield (%)
    "US2Y": "^IRX",        # 2-Year Treasury Yield (%)
    "TLT": "TLT",          # 20+ Year Treasury ETF
    "IEF": "IEF",          # 7-10 Year Treasury ETF
    "BND": "BND",          # Total US Bond Market ETF
    "LQD": "LQD"           # Investment Grade Corporate Bond ETF
}

bonds = fetch_data(list(bond_tickers.values()))
bonds.columns = bond_tickers.keys()
bonds.head()

,US10Y,US2Y,TLT,IEF,BND,LQD
Date,,,,,,
2010-01-04,3.841,0.055,57.187580,62.727306,49.618130,58.978405
2010-01-05,3.755,0.060,57.556885,63.002720,49.763191,59.260059
2010-01-06,3.808,0.045,56.786392,62.748470,49.744297,59.085476
2010-01-07,3.822,0.045,56.881927,62.748470,49.706451,59.158672
2010-01-08,3.808,0.040,56.856415,62.826138,49.756901,59.288269


In [43]:
# Leading Stocks by Sector
stock_tickers = {
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Google": "GOOGL",
    "Amazon": "AMZN",
    "Meta": "META",
}

stocks = fetch_data(list(stock_tickers.values()))
stocks.columns = stock_tickers.keys()
stocks.head()

,Apple,Microsoft,Google,Amazon,Meta
Date,,,,,
2010-01-04,6.424603,23.173353,15.576998,6.6950,NaN
2010-01-05,6.435713,23.180834,15.508403,6.7345,NaN
2010-01-06,6.333344,23.038572,15.117455,6.6125,NaN
2010-01-07,6.321636,22.798975,14.765527,6.5000,NaN
2010-01-08,6.363664,22.956207,14.962369,6.6760,NaN


In [44]:
# Merge all classes on Date
market_data = (
    indexes
    .join(bonds, how="outer")
    .join(stocks, how="outer")
)

# Fill and clean
market_data = market_data.sort_index().bfill() #fillna(method="bfill")
market_data.head(10)
market_data.info(10)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4122 entries, 2010-01-04 to 2025-11-04
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   S&P500     4121 non-null   float64
 1   NASDAQ     4122 non-null   float64
 2   DowJones   4121 non-null   float64
 3   CAC40      4122 non-null   float64
 4   DAX        4122 non-null   float64
 5   FTSE100    4122 non-null   float64
 6   Nikkei225  4122 non-null   float64
 7   HangSeng   4122 non-null   float64
 8   MSCIWorld  4122 non-null   float64
 9   US10Y      4122 non-null   float64
 10  US2Y       4122 non-null   float64
 11  TLT        4122 non-null   float64
 12  IEF        4122 non-null   float64
 13  BND        4122 non-null   float64
 14  LQD        4122 non-null   float64
 15  Apple      4122 non-null   float64
 16  Microsoft  4122 non-null   float64
 17  Google     4122 non-null   float64
 18  Amazon     4122 non-null   float64
 19  Meta       4122 non-null   flo

In [45]:
# Save files
market_data.to_csv(OUT / "Market_Prices_All.csv")
print("Saved macro CSVs to:", OUT)

Saved macro CSVs to: C:\Users\dax_a\ml_outputs


In [53]:
returns = market_data.pct_change(fill_method=None).dropna()
# ---------------------------------------------------------------
# SAVE ALL RETURNS TO CSV
# ---------------------------------------------------------------
returns.to_csv(OUT / "Market_Returns_All.csv")

In [54]:
# ===============================================================
# MERGING DATASETS
# ===============================================================

from pathlib import Path
OUT = Path.home() / "ml_outputs"
market = pd.read_csv(OUT / 'Market_Returns_All.csv', index_col=0, parse_dates=True)
macro = pd.read_csv(OUT / 'Macro_AllSources.csv', index_col=0, parse_dates=True)

In [56]:
# Merge market and macroeconomic data
data = market.join(macro, how='left').ffill()
print("OK Combined dataset shape:", data.shape)
data.tail(5)
data.to_csv(OUT / "Full_dataset_reference.csv")

OK Combined dataset shape: (4120, 37)


In [61]:
# ===============================================================
# CREATE BINARY TARGET (Direction)
# ===============================================================

# Example index (can change to CAC40 or others)
target_index = "Apple" if "Apple" in data.columns else data.columns[0]

# Create returns and technical indicators
data["Return"] = data[target_index] #.pct_change()
data["Volatility_20d"] = data["Return"].rolling(20).std()
data["MA20"] = data[target_index].rolling(20).mean()
data["MA50"] = data[target_index].rolling(50).mean()
data["Momentum"] = data[target_index] / data[target_index].shift(5) - 1

# Relative Strength Index (RSI)
def compute_RSI(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window).mean()
    RS = gain / loss
    return 100 - (100 / (1 + RS))

data["RSI"] = compute_RSI(data[target_index])

# Define binary target variable: 1 = Up, 0 = Down
data["Direction"] = (data["target_index"].shift(-1) > 0).astype(int)

# Drop rows with NaN
data = data.dropna()
print("OK Final dataset after feature engineering:", data.shape)

data.tail(20)

OK Final dataset after feature engineering: (3970, 45)


,S&P500,NASDAQ,DowJones,CAC40,DAX,FTSE100,Nikkei225,HangSeng,MSCIWorld,US10Y,...,OECD_CPI_2015idx_USA,OECD_Unemp_rate_pct_USA,target_index,Volatility_20d,MA20,MA50,Momentum,RSI,Direction,Return
Date,,,,,,,,,,,,,,,,,,,,,
2025-10-07,-0.003811,-0.006683,-0.001970,0.000385,0.000307,0.000475,0.000128,-0.004760,-0.005392,-0.008409,...,136.439499,4.3,-0.000818,0.015863,0.004641,0.004044,-2.040649,48.828384,1,-0.000818
2025-10-08,0.005828,0.011191,-0.000026,0.010694,0.008667,0.006886,-0.004502,0.000000,0.004928,0.000485,...,136.439499,4.3,0.006160,0.013274,0.006562,0.004378,0.912933,52.932148,0,0.006160
2025-10-09,-0.002756,-0.000814,-0.005222,-0.002329,0.000574,-0.004137,0.017711,-0.002865,-0.004632,0.004601,...,136.439499,4.3,-0.015578,0.014019,0.005068,0.004208,-3.368647,35.939620,0,-0.015578
2025-10-10,-0.027112,-0.035623,-0.018957,-0.015341,-0.015025,-0.008613,-0.010120,-0.017279,-0.024470,-0.023385,...,136.439499,4.3,-0.034522,0.016238,0.002464,0.004017,-10.973956,28.097553,1,-0.034522
2025-10-13,0.015599,0.022076,0.012928,0.002054,0.006042,0.001634,-0.025816,-0.015247,0.013749,0.000000,...,136.439499,4.3,0.009744,0.016199,0.002389,0.004116,-2.890425,54.707976,1,0.009744
2025-10-14,-0.001564,-0.007619,0.004404,-0.001845,-0.006191,0.001048,0.000000,-0.017309,0.000664,-0.007159,...,136.439499,4.3,0.000444,0.016179,0.002105,0.004167,-1.542933,52.447046,1,0.000444
2025-10-15,0.004026,0.006588,-0.000371,0.019872,-0.002293,-0.002962,0.017618,0.018444,0.004094,0.005967,...,136.439499,4.3,0.006336,0.016205,0.002246,0.003276,0.028606,46.304983,0,0.006336
2025-10-16,-0.006294,-0.004744,-0.006509,0.013816,0.003756,0.001199,0.012692,-0.000853,-0.003471,-0.017301,...,136.439499,4.3,-0.007580,0.016283,0.002099,0.002489,-0.513409,49.299155,1,-0.007580
2025-10-17,0.005271,0.005205,0.005187,-0.001757,-0.018177,-0.008637,-0.014408,-0.024776,0.004202,0.007797,...,136.439499,4.3,0.019559,0.015285,0.001475,0.002033,-1.566579,56.746128,1,0.019559


In [63]:
data.to_csv(OUT / "Full_dataset_reference.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3970 entries, 2010-07-29 to 2025-11-03
Data columns (total 45 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   S&P500                             3970 non-null   float64
 1   NASDAQ                             3970 non-null   float64
 2   DowJones                           3970 non-null   float64
 3   CAC40                              3970 non-null   float64
 4   DAX                                3970 non-null   float64
 5   FTSE100                            3970 non-null   float64
 6   Nikkei225                          3970 non-null   float64
 7   HangSeng                           3970 non-null   float64
 8   MSCIWorld                          3970 non-null   float64
 9   US10Y                              3970 non-null   float64
 10  US2Y                               3970 non-null   float64
 11  TLT                                397